---
title: "React Three Fiber with Theater.js"
author: "Vahram Poghosyan"
date: "2024-12-26"
categories: ["Three.js", "R3F", "Theater.js", "Visualization", "JavaScript", "TypeScript"]
format:
  html:
    code-fold: false
toc-depth: 4
jupyter: python3
highlight-style: github
include-after-body:
  text: |
    <script type="application/javascript" src="../../../javascript/light-dark.js"></script>
---

# Foreword

React Three Fiber is a React wrapper for Three.js, which allows us to use Three.js in a React application. It provides a way to create and manage 3D scenes using React components, making it easier to integrate 3D graphics into React applications. In this post, we will explore how to use React Three Fiber with [Theater.js](https://www.theatrejs.com/), a library for creating animations and interactive experiences in the browser. Theater.js basically provides a rich CG editor (like Blender) in the browser which lets us translate/scale or otherwise transform 3D objects, use animation curves, and keyframes, etc. 

## Overview of Theater.js

Theater.js is composed of two main parts and many optional extensions.

1. Core: The core of Theater.js is a JavaScript library that provides the runtime for Theater.js code (which is just a wrapper on top of Three.js, which is, by extension, a wrapper of WebGL).

    ```bash
    npm install @theatre/core
    ```
    In fact, when we use Theater to update the position of a 3D object, we explicitly set up a listener in Theater.js that picks up the changes and passes them down to Three.js.

2. Studio: The Studio is a web-based editor that allows users to create and edit Theater.js projects visually. It provides a user interface for manipulating 3D objects, creating animations, and managing scenes. We take this library out in the production version of our app (hence we may only install it as a dev dependency).
    ```bash
    npm install @theatre/studio
    ```
3. Extensions: There are many extensions that let us customize our workflow! For example, this one is a set of UI tools that enhance the functionality of the studio library. It includes the translation tools, rotation tools, and scaling tools that bringing the experience much closer to that of using a 3D editor (like Blender).